Import libraries

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, concatenate, Dropout, TextVectorization, LSTM, BatchNormalization, Activation, Lambda, MultiHeadAttention, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2, l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import backend as K

Load dataset

In [2]:
pd.set_option('display.max_columns', None)
#df = pd.read_csv('Merged_df.csv', index_col=0, low_memory=False)
df = pd.read_csv("/kaggle/input/merged-df/Merged_df.csv", index_col=0, low_memory=False)

Drop unnecessary columns

In [3]:
df1 = df.drop(['MovieID', 'Timestamp', 'Title', 'Genres', 'ZipCode', 'Movie_Title',
               'status', 'backdrop_path', 'homepage', 'imdb_id', 'poster_path', 'production_companies',
               'production_countries', 'title_year', 'year', 'original_language', "release_date", "original_title", "spoken_languages"], axis=1)

Drop rows with NA values

In [4]:
df1.dropna(inplace=True)

Converting Years into Decades (Buckets to make it a categorical variable)


In [5]:
# Convert years into decades
df1['Decade'] = (df1['Movie_Year'] // 10) * 10  # This floors each year to the start of its decade

# Now drop the original 'Movie_Year' column if it's no longer needed
df1 = df1.drop(['Movie_Year'], axis=1)

Convert 'adult' column to integers

In [6]:
# Convert 'adult' column to integers (1 for True, 0 for False)
df1['adult'] = df1['adult'].astype(int)

Convert movieids to integers

In [7]:
df1['id'] = df1['id'].astype(int)

Encode Catgorical features & Scale Numerical Variables

In [8]:
# Define categorical and numerical features
categorical_features = ['Gender', 'Occupation', 'Age', 'Decade']
numerical_features = ['vote_average', 'vote_count', 'revenue', 'runtime', 'budget', 'popularity']

# One-Hot Encoding for categorical variables
one_hot_encoder = OneHotEncoder(sparse=False)  # Using sparse=False to get a dense array
encoded_features = one_hot_encoder.fit_transform(df1[categorical_features])

# Normalize Numerical Features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df1[numerical_features])

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Convert Genres into Multiple Binary Columns

In [9]:
# Since genres are separated by commas in dataset
df1['genres'] = df1['genres'].apply(lambda x: x.split(', '))

mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df1['genres'])

Before concatenating, Drop all original variables which have been transformed

In [10]:
df1.drop(categorical_features, axis=1, inplace=True)
df1.drop(numerical_features, axis=1, inplace=True)
df1.drop("genres", axis=1, inplace=True)

Combine all processed features back into one dataframe

In [11]:
# Convert encoded categorical features, scaled numerical features, and multiple genere features back to DataFrame 
encoded_cats = pd.DataFrame(encoded_features, columns=one_hot_encoder.get_feature_names_out(categorical_features), index=df1.index)
scaled_nums = pd.DataFrame(scaled_features, columns=numerical_features, index=df1.index)
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_, index=df1.index)

# Combine all features
df_combined = pd.concat([df1, encoded_cats, scaled_nums,genres_df], axis=1)

Split into train and test dataset

In [12]:
df_train_combined, df_test_combined = train_test_split(df_combined, test_size=0.2, random_state=42)

Create User embeddings

In [13]:
num_users = df_train_combined['UserID'].max() + 1  # Add 1 to handle index error as python indexes from 0, then index 0 will be unused/reserved

# UsedID embeddings
user_id_input = Input(shape=(1,), name='user_id_input')  # Entry point for User IDs  --> Each input is a single value

# Aim to capture 10 latent characteristics of each user (10 dimensions)
# input_dim is the total number of unique users
user_embedding = Embedding(input_dim=num_users, output_dim=10, name='user_embedding')(user_id_input) # Transforms each userID into a dense vector of size 10
user_flatten = Flatten(name='user_flatten')(user_embedding) # Flatten to a 1D tensor

User demographics

In [14]:
demographic_features = ['Gender_F', 'Gender_M', 'Occupation_0', 'Occupation_1', 'Occupation_2',
                        'Occupation_3', 'Occupation_4', 'Occupation_5', 'Occupation_6',
                        'Occupation_7', 'Occupation_8', 'Occupation_9', 'Occupation_10',
                        'Occupation_11', 'Occupation_12', 'Occupation_13', 'Occupation_14',
                        'Occupation_15', 'Occupation_16', 'Occupation_17', 'Occupation_18',
                        'Occupation_19', 'Occupation_20', 'Age_1', 'Age_18', 'Age_25', 'Age_35',
                        'Age_45', 'Age_50', 'Age_56']

# User demographics input
user_demographics_input = Input(shape=(len(demographic_features),), name='user_demographics_input')

Create Movie Embeddings

In [15]:
num_movies = df_train_combined['id'].max() + 1 # Add 1 to handle index error as python indexes from 0, then index 0 will be unused/reserved

# MovieID embeddings
movie_id_input = Input(shape=(1,), name='movie_id_input')  # Entry point for Movie IDs  --> Each input is a single value

# Aim to capture 50 latent characteristics of each movie due to large number of unique movies (50 dimensions)
# input_dim is the total number of unique movies
movie_embedding = Embedding(input_dim=num_movies, output_dim=50, name='movie_embedding')(movie_id_input)
movie_flatten = Flatten(name='movie_flatten')(movie_embedding)

Movie Features

In [16]:
movie_features_columns = ['vote_average', 'vote_count', 'revenue', 'runtime', 'budget', 'popularity',
                          'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
                          'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
                          'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western', 'Decade_1910',
                          'Decade_1920', 'Decade_1930', 'Decade_1940', 'Decade_1950', 'Decade_1960',
                          'Decade_1970', 'Decade_1980', 'Decade_1990', 'Decade_2000']

num_movie_features = len(movie_features_columns)

movie_input = Input(shape=(num_movie_features,), name='movie_input')

Utilize 'Overview' and 'tagline' columns to process text data

In [21]:
# preprocessing step --> transforms strings into numerical representation of fixed length 
# Define TextVectorization for overview column
vectorizer_overview = TextVectorization(max_tokens=10000, output_sequence_length=70) # Vectorizer only considers top 10000 most common words in dataset
vectorizer_overview.adapt(df_train_combined['overview'])

# Define TextVectorization for tagline column
vectorizer_tagline = TextVectorization(max_tokens=1000, output_sequence_length=15)
vectorizer_tagline.adapt(df_train_combined['tagline'])

# Text inputs
text_input_overview = Input(shape=(1,), dtype="string", name="overview_input")
text_input_tagline = Input(shape=(1,), dtype="string", name="tagline_input")

# Apply vectorization
overview_vectors = vectorizer_overview(text_input_overview) # Tokenize input and convert each token into numeric indices based on its internal vocabulary
tagline_vectors = vectorizer_tagline(text_input_tagline)

# Embedding layers
overview_embedding = Embedding(input_dim=10000, output_dim=50, name='overview_embedding')(overview_vectors)
tagline_embedding = Embedding(input_dim=1000, output_dim=50, name='tagline_embedding')(tagline_vectors)

# # LSTM
# overview_processed = LSTM(32, name='overview_lstm')(overview_embedding)  # 32 is the number of LSTM units in the layer. Output will be a 32-D vector. 
# tagline_processed = LSTM(32, name='tagline_lstm')(tagline_embedding)

# Embedding layers followed by Multi-Head Attention
def transformer_encoder(embedding_layer):
    # Apply Multi-Head Attention using 2 heads
    attention_output = MultiHeadAttention(num_heads=2, key_dim=50)(embedding_layer, embedding_layer)
    
    # Add a feed-forward network (FFN) as commonly used in Transformers
    ffn_output = Dense(50, activation='relu')(attention_output)

    # Layer Normalization and Residual (skip) Connection
    output = LayerNormalization()(ffn_output + embedding_layer) 
    return output

# Process both text inputs with Multi-Head Attention
overview_attention = transformer_encoder(overview_embedding)
tagline_attention = transformer_encoder(tagline_embedding)

# Since attention outputs a sequence, reduce it to a single vector via global pooling or by taking the mean
overview_pooled = Lambda(lambda x: K.mean(x, axis=1))(overview_attention)
tagline_pooled = Lambda(lambda x: K.mean(x, axis=1))(tagline_attention)

Combine model inputs

In [22]:
# Combine user and movie inputs
combined_inputs = concatenate([user_flatten, user_demographics_input, movie_flatten, movie_input, overview_pooled, tagline_pooled])

Assemble Complete Model

In [23]:
# Add fully connected layers
fc1 = Dense(256, activation='relu')(combined_inputs) 
dropout1 = Dropout(0.25)(fc1)

fc2 = Dense(128, activation='relu')(dropout1)
dropout2 = Dropout(0.25)(fc2)

fc3 = Dense(64, activation='relu')(dropout2)
output = Dense(1, activation='sigmoid')(fc3) 
output =Lambda(lambda x: x * 5)(output)

# Finalizing the model
model = Model(inputs=[user_id_input, user_demographics_input, movie_id_input, movie_input, text_input_overview, text_input_tagline], outputs=output)
lr_schedule = ExponentialDecay(initial_learning_rate=0.004, decay_steps=10000, decay_rate=0.9) # Learning Rate Scheduling

# Compiling the model
model.compile(optimizer=Adam(lr_schedule), loss='mean_squared_error')

# Model summary to check the architecture
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ overview_input      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tagline_input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_vectorization… │ (None, 70)        │          0 │ overview_input[0… │
│ (TextVectorization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_vectorization… │ (None, 15)        │          0 │ tagline_input[0]… │
│ (TextVectorization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ overview_embedding  │ (None, 70, 50)    │    500,000 │ text_vectorizati… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tagline_embedding   │ (None, 15, 50)    │     50,000 │ text_vectorizati… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 70, 50)    │     20,350 │ overview_embeddi… │
│ (MultiHeadAttentio… │                   │            │ overview_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 15, 50)    │     20,350 │ tagline_embeddin… │
│ (MultiHeadAttentio… │                   │            │ tagline_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 70, 50)    │      2,550 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 15, 50)    │      2,550 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_id_input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_id_input      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 70, 50)    │          0 │ dense[0][0],      │
│                     │                   │            │ overview_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 15, 50)    │          0 │ dense_1[0][0],    │
│                     │                   │            │ tagline_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 10)     │     60,410 │ user_id_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_embedding     │ (None, 1, 50)     │ 32,727,650 │ movie_id_input[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 70, 50)    │        100 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 33,483,133 (127.73 MB)

 Trainable params: 33,483,133 (127.73 MB)

 Non-trainable params: 0 (0.00 B)

Preparing Inputs for training

In [24]:
# Extract ratings (Y-values)
ratings_train = df_train_combined['Rating'].values
ratings_test = df_test_combined['Rating'].values

# Prepare inputs for training
user_ids_train = df_train_combined['UserID'].values
movie_ids_train = df_train_combined['id'].values
user_demographics_train = df_train_combined[demographic_features].values
movie_features_train = df_train_combined[movie_features_columns].values
overview_train = df_train_combined['overview']  
tagline_train = df_train_combined['tagline'] 

# Prepare inputs for testing (evaluation)
user_ids_test = df_test_combined['UserID'].values
movie_ids_test = df_test_combined['id'].values
user_demographics_test = df_test_combined[demographic_features].values
movie_features_test = df_test_combined[movie_features_columns].values
overview_test = df_test_combined['overview']  
tagline_test = df_test_combined['tagline']  

Train the Model

In [25]:
history = model.fit([user_ids_train, user_demographics_train, movie_ids_train, movie_features_train, overview_train, tagline_train], ratings_train,
                    validation_data=([user_ids_test, user_demographics_test, movie_ids_test, movie_features_test, overview_test, tagline_test], ratings_test),
                    epochs=5,
                    batch_size=128)

Epoch 1/5
5953/5953 ━━━━━━━━━━━━━━━━━━━━ 199s 32ms/step - loss: 0.9076 - val_loss: 0.8132
Epoch 2/5
5953/5953 ━━━━━━━━━━━━━━━━━━━━ 189s 32ms/step - loss: 0.7945 - val_loss: 0.7822
Epoch 3/5
5953/5953 ━━━━━━━━━━━━━━━━━━━━ 189s 32ms/step - loss: 0.7580 - val_loss: 0.7722
Epoch 4/5
5953/5953 ━━━━━━━━━━━━━━━━━━━━ 189s 32ms/step - loss: 0.7305 - val_loss: 0.7556
Epoch 5/5
5953/5953 ━━━━━━━━━━━━━━━━━━━━ 189s 32ms/step - loss: 0.7152 - val_loss: 0.7517


Extract Unseen Movies

In [ ]:
# Function to extract movies yet to be seen by a user
def get_unseen_movies(user_id, data = df_train_combined):
    seen_movieIDs = data[data['UserID'] == user_id]['id'].unique()
    all_movies = data["id"].unique()
    unseen_movieIDs = np.sort(np.setdiff1d(all_movies, seen_movieIDs))
    return unseen_movieIDs

Predict Ratings with titles for a particular user

In [ ]:
def predict_ratings_for_user_with_titles(user_id, num_recommendations=5, model = model, data = df_train_combined):

    # Extract unseen movie IDs for the user
    movie_id_input = get_unseen_movies(user_id, data)
    
    # Extract user IDs
    user_id_input = np.array([user_id] * len(movie_id_input))
    
    # Extract dataset for this particular user and just keep first row as features will be constant
    user_df = data[data['UserID'] == user_id][:1]

    # Extract users demographic features and repeat for the length of unseen movies
    user_demographics_pred = np.repeat(user_df[demographic_features].values, len(movie_id_input), axis=0)

    # Extract dataset on movie features for unseen movies
    unseen_movies_df = data[data['id'].isin(movie_id_input)]
    # Drop duplicates in movieIDs as movie features will be constant and we only want the movie features
    unseen_movies_df = unseen_movies_df.drop_duplicates(subset='id', keep='first')
    # Sort by MovieID to ensure same order as the movie_id_input array
    unseen_movies_df = unseen_movies_df.sort_values(by='id', ascending=True)
    
    # Extract Movie features input
    movie_features_input = unseen_movies_df[movie_features_columns].values
    
    # Extract Movie Overview input
    overview_input = unseen_movies_df["overview"].values
    
    # Extract Movie Tagline input
    tagline_input = unseen_movies_df["tagline"].values

    # Predict Ratings for unseen movies
    predicted_ratings = model.predict([user_id_input, user_demographics_pred, movie_id_input, 
                                       movie_features_input, overview_input, tagline_input])
    
    unseen_movies_df["predicted_ratings"] = predicted_ratings    
    
    unseen_movies_df.sort_values(by='predicted_ratings', ascending=False, inplace = True)
    
    final_movies_df = unseen_movies_df[["title", "predicted_ratings"]].head(num_recommendations)

    return final_movies_df

In [ ]:
predict_ratings_for_user_with_titles(67,10, data = df_combined)